# Dask Delayed

Материалы:
* Макрушин С.В. Лекция 13: Dask Delayed
* https://docs.dask.org/en/latest/delayed.html
* JESSE C. DANIEL. Data Science with Python and Dask.


## Задачи для совместного разбора

![](https://i.imgur.com/AwiN8y6.png)
![](https://i.imgur.com/ceY6guU.png)

1. Напишите 2 функции, имитирующие CPU-bound задачу и IO-bound задачу:

`cpu_task()`: генерирует 100 тыс. случайных чисел и возвращает их сумму (без использования `numpy`)

`io_task()`: "спит" 0.1 сек, затем генерирует случайное число и возвращает его

Замерьте время выполнения 100 последовательных вызовов каждой из этих функций. Распараллелив вычисления при помощи `dask.delayed`, сократите время выполнения. Исследуйте, как зависит время вычислений от выбранного планировщика `scheduler`.

In [3]:
import random

def cpu_task():
    return sum([random.randint(0, 10) for _ in range(100_000)])

In [4]:
%%time
res = [cpu_task() for _ in range(100)]

Wall time: 19.9 s


In [7]:
import time

def io_task():
    time.sleep(0.1)
    return random.randint(0, 10)

In [8]:
%%time
res = [io_task() for _ in range(100)]

Wall time: 10.9 s


In [11]:
from dask import delayed, compute

cpu_task_delayed = delayed(cpu_task)
cpu_task_delayed().compute()

501267

In [12]:
%%time
res = [cpu_task_delayed() for _ in range(100)]
res = compute(res, scheduler='multiprocessing')

Wall time: 5.32 s


In [13]:
%%time
res = [cpu_task_delayed() for _ in range(100)]
res = compute(res, scheduler='threading')

Wall time: 25.5 s


In [14]:
io_task_delayed = delayed(io_task)

In [15]:
%%time
res = [io_task_delayed() for _ in range(100)]
res = compute(res, scheduler='multiprocessing')

Wall time: 2.76 s


In [16]:
%%time
res = [io_task_delayed() for _ in range(100)]
res = compute(res, scheduler='threading')

Wall time: 1.45 s


## Лабораторная работа 14

1. Напишите функцию, которая считывает файл формата xml из архива `reviewers_full.zip` и по данным этого файла формирует список словарей, содержащих следующие ключи: `username`, `name`, `sex`, `country`, `mail`, `registered`, `birthdate`, `name_prefix`, `country_code`. Часть из этих значений в исходном файле хранится в виде тэгов, часть - в виде атрибутов тэгов. Для конкретного человека какие-то из этих ключей могут отсутствовать. 



2. Измерьте время выполнения функции из задания 1 на всех файлах из архива. Ускорьте время выполнения, используя `dask.delayed`.

3. Задекорируйте функцию из задания 1 при помощи `dask.delayed` и создайте список `reviewers`, состоящий из 5 объектов `delayed` (по одному объекту на файл). Из списка объектов `delayed`, создайте `dask.bag` при помощи метода `db.from_delayed`. Добавьте ключ `birth_year`, в котором хранится год рождения человека. Оставьте в выборке только тех людей, которые __наверняка__ моложе 1980 года. Преобразуйте поле `id` к целому типу.

4. Из `dask.bag`, полученного в задании 3, создайте `dask.dataframe` при помощи метода `bag.to_dataframe`. Укажите столбец `id` в качестве индекса.

5. Назовем отзыв негативным, если оценка равна 0, 1 или 2. Загрузите данные о негативных отзывах из файлов архива `reviews_full` (__ЛР12__) в виде `dask.DataFrame`. Посчитайте количество отзывов с группировкой по пользователю, оставившему отзыв. Объедините результат с таблицей, полученной в задаче 4.